In [1]:
import numpy as np 
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import random

In [3]:
with pd.ExcelFile("mypairing.xlsx") as source_us :
    estimate = [pd.read_excel(source_us, i).to_numpy() for i in range(0,4)]

###Structure
##1 feuille par scénario
#lignes = nos bt (0 à 7)
#colonne = leurs bt (0 à 7)
#case = score de notre joueur (de 0 à 20)

#On se base pour le moment sur un unique estimate partagé

In [4]:
def calculate(matchs,estimate) :  #Retourne le score attendu correspondant a un ensemble de 8 matchs
    result = 0
    for s in range(0,4) :  #Unpack les senarii
        for p in range(0,2) :  #Unpack les parties par sénario
            bt_us = int(matchs[s][p][0])
            bt_them = int(matchs[s][p][1])
            result += estimate[s][bt_us,bt_them] #score attendue de notre bt match[s][p][0] contre leur bt match[s][p][1]
    return min(120,result)  #Cap a 120

In [5]:
def quick_opt_blind_us_2(l_us, l_them, matchs, estimate) : #1er algo basique pour tenter de trouver le meilleur pick (nous)
    n_tries = 100      #Nombre de combinaisons aléatoires envisagées a chaque coup
    rem = len(l_us)   
    best_pick_them = l_us[0]
    max_res = 0
    
    for np in l_us :  #On donne une chance a chaque BT restant en main
        res = 0
        for i in range(0,n_tries):  #On calcule n_tries combinaison possible si on choisit ce bt là
            m = matchs.copy()
            l_us_rest = l_us.copy()
            l_us_rest.remove(np)
            mouv_them = random.sample(l_them, k=rem)   #On prends tout les futurs autres pick au hasard...
            mouv_us = [np]+random.sample(l_us_rest, k=rem-1)   #... pour les deux joueurs
            for j in range(0,int(rem/2)):
                m.append([[mouv_us[int(0+j*2)], mouv_them[int(0+j*2)]],[mouv_us[int(1+j*2)], mouv_them[int(1+j*2)]]])
            res += calculate(m, estimate)   #On recompose le set de 8 match et on calcul son score pour chaque combinaison

        m_res = res/n_tries  #On calcule la moyenne résultante pour chaque bt proposé à ce coup
        if m_res > max_res :
            max_res = m_res
            best_pick_us = np  #On détermine, en moyenne, le meilleur BT pour ce coup
            
    return best_pick_us

In [6]:
def quick_opt_blind_opponent_2(l_us, l_them, matchs, estimate_opp) : #Voir au dessus. Même principe mais en minimisant le score (donc enmaximisant celui de l'adversaire)
    rem = len(l_us)   
    best_pick_them = l_them[0]
    min_res = 120
    
    for np in l_them :  
        res = 0
        for i in range(0,100):
            m = matchs.copy()
            l_them_rest = l_them.copy()
            l_them_rest.remove(np)
            mouv_us = random.sample(l_us, k=rem)
            mouv_them = [np]+random.sample(l_them_rest, k=rem-1)
            for j in range(0,int(rem/2)):
                m.append([[mouv_us[int(0+j*2)], mouv_them[int(0+j*2)]],[mouv_us[int(1+j*2)], mouv_them[int(1+j*2)]]])
            res += calculate(m, estimate_opp)

        m_res = res/100
        if m_res < min_res :
            min_res = m_res
            best_pick_them = np
            
    return best_pick_them
        

In [7]:
def quick_opt_blind_us_3(l_us, l_them, matchs, estimate, a_them) : #2eme algo basique pour pick le meilleur parmis leurs 2 attaquants
    n_tries = 100      #Nombre de combinaisons aléatoires envisagées a chaque coup
    rem = len(l_us)   
    best_pick_them = l_us[0]
    max_res = 0
    
    for o in a_them :  #On teste les deux possibilitées
        res = 0
        for i in range(0,n_tries):  #On calcule n_tries combinaison possible si on choisit ce bt là
            m = matchs.copy()
            l_us_rest = l_us.copy()
            l_us_rest.remove(np)
            
            
            
            
            
            mouv_them = random.sample(l_them, k=rem)   #On prends tout les futurs autres pick au hasard...
            mouv_us = [np]+random.sample(l_us_rest, k=rem-1)   #... pour les deux joueurs
            
            
            
            
            for j in range(0,int(rem/2)):
                m.append([[mouv_us[int(0+j*2)], mouv_them[int(0+j*2)]],[mouv_us[int(1+j*2)], mouv_them[int(1+j*2)]]])
            res += calculate(m, estimate)   #On recompose le set de 8 match et on calcul son score pour chaque combinaison

        m_res = res/n_tries  #On calcule la moyenne résultante pour chaque bt proposé à ce coup
        if m_res > max_res :
            max_res = m_res
            best_pick_us = np  #On détermine, en moyenne, le meilleur BT pour ce coup
            
    return best_pick_us

In [8]:
def turn12(l_us, l_them, matchs, estimate, mode): #Pour le sénario 1 ou 2
    
    #choix des défenseurs d_us 
    #et d_them
    if mode == "bot" :
        d_us = quick_opt_blind_us_2(l_us, l_them, matchs, estimate)  #Ici, on utilise le premier algo pour notre choix
        d_them = quick_opt_blind_opponent_2(l_us, l_them, matchs, estimate) #Et là, pour leur choix
    elif mode == "training" :
        print("Il nous reste :",l_us)
        print("Il leur reste :",l_them)
        d_us = int(input("Notre défenseur ? "))
        d_them = quick_opt_blind_opponent_2(l_us, l_them, matchs, estimate)
        print("Leur défenseur : "+str(d_them))
    
    #d_us = random.choice(l_us)                   #Alternative en random
    #d_them = random.choice(l_them)
    
    l_us.remove(d_us) #On retire les defenseurs proposés des BT disponibles de chaque équipe
    l_them.remove(d_them)
    
    #choix des attaquants a_us (deux) 
    #choix a_them (deux)
    if mode == "bot" :
        a_us = random.sample(l_us,k=2)                    #!!! Algo a dev
        a_them = random.sample(l_them,k=2)                #!!! Algo a dev
    elif mode == "training" :
        print("Il nous reste :",l_us)
        print("Il leur reste :",l_them)
        a_us = [int(x) for x in input("Nos attanquants ? ").split()]
        a_them = random.sample(l_them,k=2)
        print("Leurs attaquants : "+str(a_them[0])+" et "+str(a_them[1]))
    
    #choix de l'attaquant pick_us parmis les 2 a_them proposés  
    #choix de l'attaquant pick_them parmis les 2 a_us proposés  
    if mode == "bot" :
        pick_us = random.choice(a_them)                   #!!! Algo a dev
        pick_them = random.choice(a_us)                   #!!! Algo a dev
    elif mode == "training" :
        pick_us = int(input("Attaquant retenu pour eux ? "))
        pick_them = random.choice(a_us)
        print("Ils ont retenu notre attaquant : "+str(pick_them))
    
    l_us.remove(pick_them) #On retire les attaquants choisis des disponibles de chaque équipe
    l_them.remove(pick_us)
    
    match = [[d_us, pick_us],[pick_them, d_them]]
    if mode == 'training' :
        print("Résumé des matchs :")
        print(str(d_us)+" VS "+str(pick_us))
        print(str(pick_them)+" VS "+str(d_them))
    ###les matchs sont une liste(4) [1/scenario] de liste (2) [1/partie] de listes (2) [1/bt]
    ## Chaque liste(2) de partie est donnée dans l'ordre : [notre bt, leur bt]
    # Chaque tour de scénario renvoie deux parties "match" a concaténer a la liste des "matchs"
    
    return (l_us, l_them, match)

In [9]:
def turn34(l_us, l_them, matchs, estimate, mode) :
    
    #choix des défenseurs d3_us 
    #et d3_them
    
    if mode == "bot" :
        d3_us = quick_opt_blind_us_2(l_us, l_them, matchs, estimate)  #Ici, on utilise le premier algo pour notre choix
        d3_them = quick_opt_blind_opponent_2(l_us, l_them, matchs, estimate) #Et là, pour leur choix
    elif mode == "training" :
        print("Il nous reste :",l_us)
        print("Il leur reste :",l_them)
        d3_us = int(input("Notre défenseur ? "))
        d3_them = quick_opt_blind_opponent_2(l_us, l_them, matchs, estimate)
        print("Leur défenseur : "+str(d3_them))

    #d3_us = random.choice(l_us)                   #Alternative en random
    #d3_them = random.choice(l_them)
    
    l_us.remove(d3_us) #On retire les defenseurs proposés des BT disponibles de chaque équipe
    l_them.remove(d3_them)

    #choix des attaquants a_us (deux) 
    #choix a_them (deux)
    if mode == "bot" :
        a_us = random.sample(l_us,k=2)                    #!!! Algo a dev
        a_them = random.sample(l_them,k=2)                #!!! Algo a dev
    elif mode == "training" :
        print("Il nous reste :",l_us)
        print("Il leur reste :",l_them)
        a_us = [int(x) for x in input("Nos attaquants ? ").split()]
        a_them = random.sample(l_them,k=2)
        print("Leurs attaquants : "+str(a_them[0])+" et "+str(a_them[1]))
    
    l_us.remove(a_us[0]) #On retire cette fois-ci les 2 bt attaquants des disponibles de chaque équipe
    l_us.remove(a_us[1])
    l_them.remove(a_them[0])
    l_them.remove(a_them[1])
    
    d4_us = l_us[0] #Le dernier Bt restant de chaque team devient le défenseur du dernièr scénario
    d4_them = l_them[0] 
    if mode == 'training' :
        print(str(d4_us)+" est donc automatiquement le dernier défenseur pour nous")
        print(str(d4_them)+" est donc automatiquement le dernier défenseur pour eux")
    
    #choix parmis les attaquants proposés de celui qui sera sur le 3 et sur le 4eme sénario : pick_us 3 ou 4
    #pour l'adversaire pick_them 3 ou 4
    if mode == "bot" :
        [pick3_us, pick4_us] = random.sample(a_them, 2)     #!!! Algo a dev
        [pick3_them, pick4_them] = random.sample(a_us, 2)   #!!! Algo a dev
    elif mode == "training" :
        pick3_us = int(input("Parmi leurs attaquants, lequel pour affronter notre 3eme défenseur ? "))
        a_them.remove(pick3_us)
        pick4_us = int(a_them[0])
        print(str(pick4_us)+' affrontera donc notre 4eme défenseur')
        [pick3_them, pick4_them] = random.sample(a_us, 2)   #!!! Algo a dev

    #Reconstitution des matchs pour les scénario 3 et 4
    match3 = [[d3_us, pick3_us],[pick3_them, d3_them]]
    match4 = [[d4_us, pick4_us],[pick4_them, d4_them]]
    if mode == 'training' :
        print("Résumé des matchs :")
        print(str(d3_us)+" VS "+str(pick3_us))
        print(str(pick3_them)+" VS "+str(d3_them))
        print("And :")
        print(str(d4_us)+" VS "+str(pick4_us))
        print(str(pick4_them)+" VS "+str(d4_them))   
    
    return match3, match4

In [10]:
def pairing(estimate, mode) :

    l_us = list(range(0,8)) #bt restants pour nous 
    l_them = list(range(0,8)) #bt restants pour eux
    matchs = []  #Initialisation du set des matchs à jouer

    l_us, l_them, pairing = turn12(l_us, l_them, matchs, estimate, mode) #Tour 1 
    matchs.append(pairing) #On concatene les resultats

    l_us, l_them, pairing = turn12(l_us, l_them, matchs, estimate, mode) #Tour 2
    matchs.append(pairing) #On concatene les resultats
    
    match3, match4 = turn34(l_us, l_them, matchs, estimate, mode) #Tour 3 + 4
    matchs.append(match3)
    matchs.append(match4)  
    
    return calculate(matchs, estimate) #Calcule du score d'équipe
    

In [11]:
### Cette partie est là pour des tests non-interactifs

# res = []  #Liste de tout les scores d'équipes

# for t in range(0,100) :  #Nombre d'essai de pairing
#     res.append(pairing(estimate,"bot"))  #Calcule du score d'équipe
    
# r = np.asarray(res)
# print(r.mean(), r.max(), r.min(), r.std())
    
# plt.hist(res,bins=120)  #Distribution des scores d'équipes
# plt.show()

# ¤¤¤ Démarage ¤¤¤

Pour démarer, rentrez le nom de mode : training ou assistant

In [12]:
# main 
mode = str(input("Mode : "))
resultat = pairing(estimate, mode)
print("Le résultat estimé pour l'équipe est de : "+str(resultat))

Mode : training
Il nous reste : [0, 1, 2, 3, 4, 5, 6, 7]
Il leur reste : [0, 1, 2, 3, 4, 5, 6, 7]
Notre défenseur ? 0
Leur défenseur : 6
Il nous reste : [1, 2, 3, 4, 5, 6, 7]
Il leur reste : [0, 1, 2, 3, 4, 5, 7]
Nos attanquants ? 1 2
Leurs attaquants : 2 et 3
Attaquant retenu pour eux ? 2
Ils ont retenu notre attaquant : 1
Résumé des matchs :
0 VS 2
1 VS 6
Il nous reste : [2, 3, 4, 5, 6, 7]
Il leur reste : [0, 1, 3, 4, 5, 7]
Notre défenseur ? 2
Leur défenseur : 0
Il nous reste : [3, 4, 5, 6, 7]
Il leur reste : [1, 3, 4, 5, 7]
Nos attanquants ? 3 4
Leurs attaquants : 5 et 1
Attaquant retenu pour eux ? 5
Ils ont retenu notre attaquant : 3
Résumé des matchs :
2 VS 5
3 VS 0
Il nous reste : [4, 5, 6, 7]
Il leur reste : [1, 3, 4, 7]
Notre défenseur ? 4
Leur défenseur : 7
Il nous reste : [5, 6, 7]
Il leur reste : [1, 3, 4]
Nos attaquants ? 5 6
Leurs attaquants : 4 et 3
7 est donc automatiquement le dernier défenseur pour nous
1 est donc automatiquement le dernier défenseur pour eux
Parmi leu